In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.mixed_precision import experimental as mixed_precision

# Enable mixed precision
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

# Define the path to the dataset and batch size
path = "aerospace_images"
batch_size = 16  # Increased batch size

# Step 1: Set up data generators
image_generator = ImageDataGenerator(rescale=1/255, 
                                     horizontal_flip=True,
                                     zoom_range=0.2, 
                                     validation_split=0.2)

image_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                    directory=path,
                                                    shuffle=True,
                                                    target_size=(224, 224),
                                                    subset="training",
                                                    class_mode="categorical")

validation_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                         directory=path,
                                                         shuffle=True,
                                                         target_size=(224, 224),
                                                         subset="validation",
                                                         class_mode="categorical")

IMG_SHAPE = (224, 224, 3)
num_of_categories = 8  # Assuming this is the correct number of categories

# Step 2: Load a pre-trained VGG19 model without the top layer
base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE, 
                                         include_top=False, 
                                         weights='imagenet')

# Freeze all layers of the base model
base_model.trainable = False



Found 6227 images belonging to 8 classes.
Found 1554 images belonging to 8 classes.


In [10]:
# Step 3: Build the model with a custom top that includes dropout
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),  # Dropout layer for regularization
    tf.keras.layers.Dense(num_of_categories, activation='softmax', dtype=tf.float32)  # Ensure output is float32
])

model._name = "Air_VGG19"



In [11]:
# Step 4: Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Print the model summary
print("Initial model summary (with frozen layers):")
print(model.summary())

# Step 6: Train the model with frozen base layers
hist_initial = model.fit(
    image_dataset,
    epochs=6,  # Reduced epochs for initial training
    validation_data=validation_dataset,
    verbose=1
)

Initial model summary (with frozen layers):


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ ?                      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,024,384 (76.39 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,024,384 (76.39 MB)

None
Epoch 1/6
390/390 ━━━━━━━━━━━━━━━━━━━━ 2091s 5s/step - accuracy: 0.2933 - loss: 1.7448 - val_accuracy: 0.6165 - val_loss: 1.2972
Epoch 2/6
 91/390 ━━━━━━━━━━━━━━━━━━━━ 21:20 4s/step - accuracy: 0.5595 - loss: 1.3482

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: [Errno 2] No such file or directory: 'aerospace_images\\airliner\\10276.jpg'
Traceback (most recent call last):

  File "C:\Users\Rajarshi\anaconda3\envs\py312\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\Rajarshi\anaconda3\envs\py312\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\Rajarshi\anaconda3\envs\py312\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\Rajarshi\anaconda3\envs\py312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_

In [5]:
# Step 7: Unfreeze some top layers of the base model for fine-tuning
for layer in base_model.layers[-4:]:  # Adjust the number of layers to unfreeze as needed
    layer.trainable = True

# Recompile the model after unfreezing
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary after unfreezing layers
print("Model summary after unfreezing some layers:")
print(model.summary())

# Step 8: Continue training the model with fine-tuning
hist_fine_tune = model.fit(
    image_dataset,
    epochs=10,  # Additional epochs for fine-tuning, reduced for speed
    validation_data=validation_dataset,
    verbose=1
)


Model summary after unfreezing some layers:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ ?                      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,024,384 (76.39 MB)

 Trainable params: 7,079,424 (27.01 MB)

 Non-trainable params: 12,944,960 (49.38 MB)

None
Epoch 1/10


C:\Users\Rajarshi\anaconda3\envs\py312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 55/390 ━━━━━━━━━━━━━━━━━━━━ 27:47 5s/step - accuracy: 0.1600 - loss: 2.5201

KeyboardInterrupt: 

In [ ]:
model.save("C:\Users\Rajarshi\Downloads\Programs\air_vgg19.h5")